In [63]:
import pandas as pd 
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import LabelEncoder

In [2]:
data= pd.read_csv("D:\PFE\Fact_Table_Rev.csv", low_memory=False)

In [3]:
data

,montant_HT,montant_ttc,date_saisie,date_facture,Fk_prestation,Fk_famille_prestation,Fk_nationnalite,Fk_Societe,Fk_typesociete
0,4.0,3.571429,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
1,6.0,5.357143,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
2,8.0,7.142857,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
3,10.0,8.928571,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
4,8.5,7.589286,2006-09-18,2006-09-18,BOI,BAR,TN,STANDART,SOC
...,...,...,...,...,...,...,...,...,...
937562,10.5,9.375000,2013-09-24,2013-09-24,BOI,BAR,LI,GRINPARAPH,LAB
937563,7.0,6.250000,2013-09-24,2013-09-24,BOI,BAR,LI,GRINPARAPH,LAB
937564,3.5,3.125000,2013-09-24,2013-09-24,BOIR,CAVE,TN,STANDART,SOC
937565,196.0,175.000000,2013-09-24,2013-09-24,REP,RESTO,TN,STANDART,SOC


In [145]:
df = pd.DataFrame(data, columns=["montant_HT", "date_saisie", "Fk_famille_prestation","Fk_typesociete"])

In [146]:
df['date_saisie'] = pd.to_datetime(df['date_saisie'])

In [147]:
df

,montant_HT,date_saisie,Fk_famille_prestation,Fk_typesociete
0,4.0,2006-09-18,BAR,SOC
1,6.0,2006-09-18,BAR,SOC
2,8.0,2006-09-18,BAR,SOC
3,10.0,2006-09-18,BAR,SOC
4,8.5,2006-09-18,BAR,SOC
...,...,...,...,...
937562,10.5,2013-09-24,BAR,LAB
937563,7.0,2013-09-24,BAR,LAB
937564,3.5,2013-09-24,CAVE,SOC
937565,196.0,2013-09-24,RESTO,SOC


In [148]:
print(df)

        montant_HT date_saisie Fk_famille_prestation Fk_typesociete
0              4.0  2006-09-18                   BAR            SOC
1              6.0  2006-09-18                   BAR            SOC
2              8.0  2006-09-18                   BAR            SOC
3             10.0  2006-09-18                   BAR            SOC
4              8.5  2006-09-18                   BAR            SOC
...            ...         ...                   ...            ...
937562        10.5  2013-09-24                   BAR            LAB
937563         7.0  2013-09-24                   BAR            LAB
937564         3.5  2013-09-24                  CAVE            SOC
937565       196.0  2013-09-24                 RESTO            SOC
937566         3.5  2013-09-24                  CAVE            PET

[937567 rows x 4 columns]


In [149]:
df= df.drop(index=df.loc[(df['date_saisie'] < '2010-01-01') | (df['date_saisie'] > '2019-12-31')].index)

In [150]:
df.reset_index(drop=True, inplace=True)

In [141]:
print(df)

        montant_HT date_saisie Fk_famille_prestation Fk_typesociete
0              7.0  2010-01-01                   BAR            SOC
1             21.5  2010-01-01                   BAR            SOC
2              8.0  2010-01-01                   BAR            SOC
3              9.5  2010-01-01                   BAR            SOC
4             10.5  2010-01-01                   BAR            SOC
...            ...         ...                   ...            ...
584661        54.0  2019-12-31                  CAVE            SOC
584662       112.0  2019-12-31                TORRET            SOC
584663        46.0  2019-12-31                TORRET            SOC
584664        22.0  2019-12-31                TORRET            SOC
584665        24.0  2019-12-31                TORRET            SOC

[584666 rows x 4 columns]


In [151]:
label_encoder = LabelEncoder()
df['Fk_prestation_code'] = label_encoder.fit_transform(df['Fk_famille_prestation'])
label_encoder = LabelEncoder()
df['Fk_typesociete_code'] = label_encoder.fit_transform(df['Fk_typesociete'])
df.head()

,montant_HT,date_saisie,Fk_famille_prestation,Fk_typesociete,Fk_prestation_code,Fk_typesociete_code
0,12.0,2010-01-01,PISC,SOC,11,5
1,198.0,2010-01-01,SEJOUR,SOC,14,5
2,5.0,2010-01-01,CAVE,SOC,2,5
3,43.0,2010-01-01,RESTO,SOC,13,5
4,7.0,2010-01-01,BAR,SOC,1,5


Linear Regression Model

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Supposons que vous avez vos données dans un DataFrame pandas appelé 'data'

# Séparation des features et de la target
X = df[['Fk_prestation_code', 'Fk_typesociete_code']]
y = df['montant_HT']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)


In [14]:
model = LinearRegression()

In [16]:
model.fit(X_train, y_train)

LinearRegression()

In [17]:
y_pred = model.predict(X_test)

In [18]:
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 94537.2044992097


In [19]:
X_train = df[(df['date_saisie'].dt.year >= 2010) & (df['date_saisie'].dt.year <= 2018)][['Fk_prestation_code', 'Fk_typesociete_code']]
X_test = df[df['date_saisie'].dt.year == 2019][['Fk_prestation_code', 'Fk_typesociete_code']]
y_train = df[(df['date_saisie'].dt.year >= 2010) & (df['date_saisie'].dt.year <= 2018)]['montant_HT']
y_test = df[df['date_saisie'].dt.year == 2019]['montant_HT']

In [20]:
model = LinearRegression()

In [21]:
model.fit(X_train, y_train)

LinearRegression()

In [22]:
y_pred = model.predict(X_test)

In [23]:
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 138040.63131902248


In [24]:
test_data = pd.concat([X_test, y_test], axis=1)

In [25]:
total_montant_HT_by_code = test_data.groupby('Fk_prestation_code')['montant_HT'].sum()

# Display the total montant_HT for each unique Fk_prestation_code
print("Total montant_HT for each unique Fk_prestation_code:")
print(total_montant_HT_by_code)

Total montant_HT for each unique Fk_prestation_code:
Fk_prestation_code
0     6.193900e+04
1     1.892159e+05
2     2.209189e+05
3     4.812000e+03
4     5.190000e+02
5     3.880193e+04
7     0.000000e+00
8     2.191949e+04
11    1.349750e+04
12    2.793753e+05
13    2.109300e+06
14    1.307727e+06
15    8.759200e+02
16    6.057224e+05
Name: montant_HT, dtype: float64


In [26]:
total_montant_HT_by_code = test_data.groupby('Fk_typesociete_code')['montant_HT'].sum()

# Display the total montant_HT for each unique Fk_prestation_code
print("Total montant_HT for each unique Fk_typesociete_code:")
print(total_montant_HT_by_code)

Total montant_HT for each unique Fk_typesociete_code:
Fk_typesociete_code
0    2.637763e+05
1    7.413500e+04
2    4.136254e+05
3    1.716000e+03
4    7.596027e+05
5    3.341769e+06
Name: montant_HT, dtype: float64


In [27]:
sum_y_pred = sum(y_pred)
print("Sum of y_pred:", sum_y_pred)

Sum of y_pred: 5141453.502138092


SARIMAX

In [152]:
df['date_saisie'] = pd.to_datetime(df['date_saisie'])
df.set_index('date_saisie', inplace=True)


In [153]:
df = df.sort_values(by='date_saisie', ascending=True)

In [154]:
index_type = type(df.index)

# Check if it's a DateTimeIndex
is_datetime_index = isinstance(df.index, pd.DatetimeIndex)

# Print the result
print("Index type:", index_type)
print("Is DateTimeIndex:", is_datetime_index)

Index type: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
Is DateTimeIndex: True


In [160]:
df.head()

,montant_HT,Fk_famille_prestation,Fk_typesociete,Fk_prestation_code,Fk_typesociete_code
date_saisie,,,,,
2010-01-01,12.0,PISC,SOC,11,5
2010-01-01,80.0,RESTO,SOC,13,5
2010-01-01,4.5,BAR,SOC,1,5
2010-01-01,4.5,BAR,SOC,1,5
2010-01-01,8.0,BAR,SOC,1,5


In [161]:
df.tail()

,montant_HT,Fk_famille_prestation,Fk_typesociete,Fk_prestation_code,Fk_typesociete_code
date_saisie,,,,,
2019-12-31,32.0,PTD,PET,12,4
2019-12-31,20.0,PTD,SOC,12,5
2019-12-31,20.0,PTD,SOC,12,5
2019-12-31,32.0,PTD,PET,12,4
2019-12-31,60.0,SEJOUR,SOC,14,5


In [163]:
train_data = df.loc[:'2018']
test_data = df.loc['2019']

In [169]:
model = SARIMAX(train_data['montant_HT'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
result = model.fit()

d:\PFE\libraires\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
d:\PFE\libraires\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [ ]:
predictions = result.predict(start='2019-01-01', end='2019-12-31', dynamic=False)

In [ ]:
rmse = mean_squared_error(test_data['montant_HT'], predictions, squared=False)
print("RMSE:", rmse)